### **Summary of Cleaning Steps**:
1. **Load the data** into Pandas.
2. **Inspect the data** for structure, summary statistics, and missing values.
3. **Check unique values** to spot inconsistencies in categorical columns.
4. **Handle missing values** by filling or dropping them.
5. **Fix inconsistent text data** by standardizing and correcting typos.
6. **Remove outliers** from the numeric columns like `Salary`.
7. **Standardize date formats** to ensure all dates are in the same format.
8. **Remove duplicate rows** from the dataset.
9. **Inspect the cleaned data** to verify your cleaning process was successful.